In [1]:
from google.colab import drive
import os
import shutil
os.chdir("/content")

drive._mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/HarukiYqM/pytorch-ZSSR

Cloning into 'pytorch-ZSSR'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 143 (delta 37), reused 35 (delta 35), pack-reused 102
Receiving objects: 100% (143/143), 2.17 MiB | 13.37 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [3]:
!pip install GPUtil

  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=a18a6c0751ce5d9179a7c249d08c3d4db0d47969a4c86752c328910b5d8abd96
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil


In [4]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1GL_Rh1N-WjrvF_-YOKOyvq0zrV6TF4hb',
                                    dest_path='./data.zip',
                                    unzip=True)
for i in os.listdir('/content/pytorch-ZSSR/set14'):
  os.remove(f'/content/pytorch-ZSSR/set14/{i}')

for i in os.listdir('/content/testing_lr_images/testing_lr_images'):
  shutil.copy(f'/content/testing_lr_images/testing_lr_images/{i}', '/content/pytorch-ZSSR/set14')

Unzipping...Done.


In [15]:
os.makedirs('/content/result')

In [5]:
os.chdir('/content/pytorch-ZSSR')
import GPUtil
import glob
import os
from utils import prepare_result_dir
import configs
from time import sleep
import sys
import run_ZSSR_single_input
import sys
import os
import configs
import ZSSR
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [6]:
conf_name = 'X2_GRADUAL_IDEAL_CONF'
gpu = None
namespace = {'conf': None}
conf = eval('configs.%s' % conf_name)
res_dir = prepare_result_dir(conf)
local_dir = '/content/pytorch-ZSSR'
files = [file_path for file_path in glob.glob('%s/*.png' % conf.input_path)
             if not file_path[-7:-4] == '_gt']

In [16]:
for file_ind, input_file in enumerate(files):
  ground_truth_file = input_file[:-4] + '_gt.png'
  if not os.path.isfile(ground_truth_file):
            ground_truth_file = '0'
  kernel_files = ['%s_%d.mat;' % (input_file[:-4], ind) for ind in range(len(conf.scale_factors))]
  kernel_files_str = ''.join(kernel_files)
  for kernel_file in kernel_files:
      if not os.path.isfile(kernel_file[:-1]):
          kernel_files_str = '0'
          break
  
  input_img, ground_truth, kernels, gpu, conf_str, results_path = input_file, ground_truth_file, kernel_files_str, gpu, conf_name, res_dir
  ground_truth = None if ground_truth == '0' else ground_truth
  print('*****', kernels)
  kernels = None if kernels == '0' else kernels.split(';')[:-1]
  conf = configs.Config()
  if conf_str is not None:
      exec ('conf = configs.%s' % conf_str)
  conf.result_path = results_path
  net = ZSSR.ZSSR(input_img, conf, ground_truth, kernels)
  device = torch.device('cuda')
  ##############weights path############################
  checkpoint = torch.load(f'/content/drive/MyDrive/Colab/VRDL/hw4/result/{input_file[-6:-4]}.pth', map_location=device)
  #######################################################
  net.model.load_state_dict(checkpoint)
  net.model = net.model.cuda()
  net.sf = np.array([3.0, 3.0])
  post_processed_output = net.final_test()
  plt.imsave(f'/content/result/{input_file[-6:-4]}_pred.png', post_processed_output, vmin=0, vmax=1)

  

***** 0
False
***** 0
False
***** 0
False
***** 0
False
***** 0
False
***** 0
False
***** 0
False
***** 0
False
***** 0
False
***** 0
False
***** 0
False
***** 0
False
***** 0
False
***** 0
False


In [13]:
img = cv2.imread('/content/results/01.png')
img.shape

(510, 510, 3)

In [9]:
img = cv2.imread('/content/testing_lr_images/testing_lr_images/01.png')
img.shape

(170, 170, 3)

In [18]:
import zipfile
def zip_compress(to_zip,save_zip_name):#save_zip_name是帶目錄的，也可以不帶就是當前目錄
#1.先判斷輸出save_zip_name的上級是否存在(判斷絕對目錄)，否則建立目錄
    save_zip_dir=os.path.split(os.path.abspath(save_zip_name))[0]#save_zip_name的上級目錄
    print(save_zip_dir)
    if not os.path.exists(save_zip_dir):
        os.makedirs(save_zip_dir)
        print('建立新目錄%s'%save_zip_dir)
    f = zipfile.ZipFile(os.path.abspath(save_zip_name),'w',zipfile.ZIP_DEFLATED)
# 2.判斷要被壓縮的to_zip是否目錄還是檔案，是目錄就遍歷操作，是檔案直接壓縮
    if not os.path.isdir(os.path.abspath(to_zip)):#如果不是目錄,那就是檔案
        if os.path.exists(os.path.abspath(to_zip)):#判斷檔案是否存在
            f.write(to_zip)
            f.close()
            print('%s壓縮為%s' % (to_zip, save_zip_name))
        else:
            print ('%s檔案不存在'%os.path.abspath(to_zip))
    else:
        if os.path.exists(os.path.abspath(to_zip)):#判斷目錄是否存在，遍歷目錄
            zipList = []
            for dir,subdirs,files in os.walk(to_zip):#遍歷目錄，加入列表
                for fileItem in files:
                    zipList.append(os.path.join(dir,fileItem))
                    # print('a',zipList[-1])
                for dirItem in subdirs:
                    zipList.append(os.path.join(dir,dirItem))
                    # print('b',zipList[-1])
            #讀取列表壓縮目錄和檔案
            for i in zipList:
                f.write(i,i.replace(to_zip,''))#replace是減少壓縮檔案的一層目錄，即壓縮檔案不包括to_zip這個目錄
                # print('%s壓縮到%s'%(i,save_zip_name))
            f.close()
            print('%s壓縮為%s' % (to_zip, save_zip_name))
        else:
            print('%s資料夾不存在' % os.path.abspath(to_zip))


zip_compress('/content/result','/content/5.zip')

/content
/content/result壓縮為/content/5.zip
